In [ ]:
# !pip install ultralytics

In [ ]:
# kütüphaneleri yüklüyoruz
from ultralytics import YOLO
import cv2
import math

In [ ]:
# kameradan video alacağız
cap = cv2.VideoCapture(0)  # kamerayı açar - 1den fazla kamera varsa o yerine kamera indeksi yazılabilir.-
cap.set(3, 1280)           # genişliği 1280 piksele ayarlar
cap.set(4, 720)            # yüksekliği 720 piksele ayarlar

In [ ]:
# YOLOv8 önceden eğitilmiş nesne tanıma modelini yüklüyoruz.
model = YOLO("yolo-Weights/yolov8n.pt")

In [ ]:
# 80 nesne katogerisini temsil ediyor
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]
# pencere boyutunu ayarla

#cv2.namedWindow('Webcam', cv2.WINDOW_NORMAL)
#cv2.resizeWindow('Webcam', 1280, 720)  # Adjust the size according to your preference


In [ ]:
# kameradan alınan görüntü ile nesne tespiti yapıp sonuçları görünüt üzerinden gösterir.

while True:   # sürekli tekrar eden döngü başlatılıyor. ç tuşuna basınca döngüden çıkar
    success, img = cap.read()  # kameradan bir görüntü karesi alınıyor.
    results = model(img, stream=True)   # alınan görüntü YOLO-v8 modeline gönderiliyor ve tespit yapılıyor

    
    for r in results:   # görüntü tespiti yapıldıkça ilerlemeye devam ediyor.
        boxes = r.boxes   # nesnenin etrafındaki kutunun bilgilerini alıyor.

        for box in boxes:  # birden fazla nesne kutu olamsı durumunda her kutu için işlem yapıyor.
            # kutuyu oluşturuyoruz
            x1, y1, x2, y2 = box.xyxy[0]  # kutunun sol üst ve sağ alt köşesinin kordinatları alınıyor.
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # kordinatlar int'a dönüştürülür

            # tespit edilen görüntünün etrafına RGB ayarlarında kutu çiziyor
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # doğruluk oranı nı yazdırıyoruz
            confidence = math.ceil((box.conf[0]*100))
            
            # doğruluk yüzdesine göre renk seçme
            if confidence > 80:
                color = (0, 255, 0)  # Yeşil
            elif confidence > 50:
                color = (255, 255, 0)  # Sarı
            else:
                color = (255, 0, 0)  # Kırmızı
                
            # Sınırlayıcı kutu çizme
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)

            # tespit edilen nesnenin sınıfınıın indeksini alıyor.
            cls = int(box.cls[0])

            # nesne detayları
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX  # yazı fontu
            fontScale = 1   # yazı boyutu
            color = (255, 0, 0)  # yazı rengi
            thickness = 2   # iki piksel kalınlığında mwtin çizwer

            # tespit edilen nesnenin sınıf adı yazdırılıyor
            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)
            cv2.putText(img, f"Dogruluk: {confidence}%", (x1, y1-20), font, fontScale, (0, 0, 255), thickness)

    # görüntüleme 
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('ç'):
        break

cap.release()
cv2.destroyAllWindows()